In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
import cv2

In [ ]:
import pandas as pd
train_images = pd.read_pickle('../input/train_images.pkl')
train_labels = pd.read_csv('../input/train_labels.csv')

img_rows = train_images.shape[1]
img_cols = train_images.shape[2]
print(train_labels.iloc[0:5])
print(train_images.shape)


In [ ]:
# Simple preprocesssing normalizing values of x to [0,1]
# Labels into 1 hot encoded matrix
num_classes=10
images = train_images.astype('float32')
images /= 255

labels = keras.utils.to_categorical(train_labels.iloc[:,1], num_classes)

In [ ]:
import matplotlib.pyplot as plt

#Let's show image with id 16
img_idx = 1

plt.title('Label: {}'.format(train_labels.iloc[img_idx]['Category']))
plt.imshow(train_images[img_idx])
cv2.imshow('')

In [ ]:
from sklearn.model_selection import train_test_split

if K.image_data_format() == 'channels_first':
    images = images.reshape(images.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    images = images.reshape(images.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=42)


    
print('x_train shape:', x_train.shape)
print('images shape:', images.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(y_train[0:5])

In [ ]:
def basic_cnn(optimizer = 'Adadelta'):
    K.clear_session()
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [ ]:
batch_size=128
epochs = 12

model = basic_cnn()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.model_selection import GridSearchCV
batch_size = [10, 20]
epochs = [10, 12]

# define the grid search parameters

optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
model = KerasClassifier(build_fn=basic_cnn, epochs=12, batch_size=128, verbose=0)
# model = KerasClassifier(build_fn=basic_cnn, verbose=1)
# param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1,cv=5)
grid_result = grid.fit(images, labels)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))